This script will make it so all images are normalized the same by taking the global min and max per band and applying it to each image.  Essentially I need to find across all training data and training bands what is the minimum and maximum values so all images are normalized the same.  I do this to avoid tiling appearences when deploying the model 

In [1]:
import pandas as pd
import numpy as np
import os

Function to get the the min-max globally but use the 5 and 95% cutoffs, I am not using this right now. 

In [5]:
#function to get files from storage bucket
def get_files(bucket_path):

    """argument is the path to where the numpy
    save files are located, return a list of filenames
    """
    all = []

    #list of files
    files = os.listdir(bucket_path)

    #get list of filenames we will use, notte I remove images that don't have a target due to clouds
    file_names = []
    for f in files:
        
        if f.endswith('.npy'):


            all.append(os.path.join(bucket_path, f))
        
    return all

def minMax(x):
    return pd.Series(index=['min','max'],data=[x.percentile(5),x.percentile(95)])


Apply the minMax function by looping through all data.  Note we have so much data I sample only 30% of it per image. 

In [8]:
#path to training, validation and testing, note these paths can change
nbac = get_files('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_nbac_subs_0_16')
mtbs = get_files('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_mtbs_subs_0_16')


final = []


def get_min_max(pathways):
    
    
    for f in pathways:

        base = os.path.basename(f)

        x = np.load(f)
        x = x.astype(float)
        x[x == 0] = np.nan
        x[x == -999] = np.nan

        #convert to dataframe
        in_shape = x.shape

        #turn to dataframe to normalize
        x = x.reshape(x.shape[0] * x.shape[1], x.shape[2])
    
        #sample 30% per image
        x = pd.DataFrame(x).dropna()#.sample(frac = 0.3)
        final.append(x)
        
    return()

get_min_max(nbac)
get_min_max(mtbs)



final4 = pd.concat(final).quantile([.01, .99])
final4
final4.reset_index().to_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_glob_norm.csv", index = False)

In [9]:
final4

,0,1,2,3,4,5,6,7,8,9,10
0.01,-239.0,-159.0,-137.0,-122.0,-269.0,-747.0,-954.0,12.0,-39.0,-26.0,1.0
0.99,245.0,1512.0,180.0,315.0,1414.0,506.0,46.0,784.0,347.0,503.0,1.0
